In [8]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings("ignore")

fase = '06_evaluar_prediccion'

In [9]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [28]:
folder = gen_config['folder']

path_prod_data_ungroup = gen_config['path_prod_data_ungroup']
path_pred_test = gen_config['path_pred_test']
path_pred_futuro = gen_config['path_pred_futuro']

path_kaggle = gen_config['path_kaggle']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
---------------------------------------06_evaluar_prediccion----------------------------------------


In [22]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [23]:
prod_data_ungroup = pd.read_parquet(f"{folder}/{path_prod_data_ungroup}")
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
df_test = pd.read_parquet(f"{folder}/{path_pred_test}")
df_fut = pd.read_parquet(f"{folder}/{path_pred_futuro}")

In [24]:
df_pred_test = df_test[['group_index', 'tn_futuro_real', 'tn_prediccion_real']]
df_pred_test['error'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])/df_pred_test['tn_futuro_real']*100
df_pred_test['error_abs'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])
df_pred_test['weight'] = df_pred_test['tn_futuro_real']/df_pred_test['tn_futuro_real'].sum()
df_pred_test.sort_values('weight', ascending=False, inplace=True)

perdida = perdida(df_pred_test['tn_futuro_real'], df_pred_test['tn_prediccion_real'])
print(f"Perdida en test (sin desagrupar): {perdida}")

Perdida en test (sin desagrupar): 0.2209635876959197


In [25]:
df_pred_ungroup = df_pred_test[['group_index', 'tn_prediccion_real']]
df_pred_ungroup = df_pred_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
df_pred_ungroup = df_pred_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

df_pred_ungroup = df_pred_ungroup[df_pred_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
df_pred_ungroup['prediccion_final'] = df_pred_ungroup['tn_prediccion_real'] * df_pred_ungroup['proporcion']
df_pred_ungroup.sort_values('product_id', ascending=True, inplace=True)

df_pred_ungroup[df_pred_ungroup['group_index'] == 281]

,group_index,product_id,tn_prediccion_real,proporcion,prediccion_final
10,281,20001,1760.481563,0.850653,1497.558138
8,281,20045,1760.481563,0.084743,149.189265
11,281,20102,1760.481563,0.010842,19.087089
9,281,20150,1760.481563,0.029031,51.108613
7,281,20261,1760.481563,0.024731,43.538457


In [26]:
df_pred_fut = df_fut[['group_index', 'tn_futuro_real', 'tn_prediccion_real']]
df_pred_fut['error'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])/df_pred_fut['tn_futuro_real']*100
df_pred_fut['error_abs'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])
df_pred_fut['weight'] = df_pred_fut['tn_futuro_real']/df_pred_fut['tn_futuro_real'].sum()
df_pred_fut.sort_values('weight', ascending=False, inplace=True)

In [27]:
df_fut_ungroup = df_fut[['group_index', 'tn_prediccion_real']]
df_fut_ungroup = df_fut_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
df_fut_ungroup = df_fut_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

df_fut_ungroup = df_fut_ungroup[df_fut_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
df_fut_ungroup['prediccion_final'] = df_fut_ungroup['tn_prediccion_real'] * df_fut_ungroup['proporcion']
df_fut_ungroup.sort_values('product_id', ascending=True, inplace=True)

In [63]:
kaggle =  df_fut_ungroup[['product_id','prediccion_final']]
kaggle['prediccion_final'] = kaggle['prediccion_final'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
#kaggle.to_csv('kaggle-new.csv', index=False)
kaggle.to_csv(f"{folder}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{path_kaggle}", index=False)


In [ ]:
#estado_control = f"06_evaluar_prediccion Terminado - {nombrefile} - {datetime.now()}"

In [ ]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n")

------------------------------------------01_LecturaDatos-------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------
